In [1]:
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pdb

In [2]:
from easydict import EasyDict as edict
from matplotlib import pyplot as plt
from torchvision import transforms as trans
from PIL import Image
from pathlib import Path
from models.Yolo_model import Yolo_model
import numpy as np
# np.seterr(all='raise')
import torch
from torch import nn, optim
# import torch.nn.functional as F
from utils.vis_utils import *
from utils.box_utils import *
from utils.dataset_tools import *
from utils.utils import *
from models.Yolo_head import Yolo_loss
from tensorboardX import SummaryWriter
from tqdm import tqdm_notebook as tqdm
from imgaug import augmenters as iaa
from Yolo_trainer_Notebook import Yolo
from torch.utils.data import DataLoader

In [3]:
conf = edict()

conf.coco_anchors = [[10, 13], [16, 30], [33, 23], [30, 61], [62, 45],
                     [59, 119], [116, 90], [156, 198], [373, 326]]
conf.train_path = Path('/home/f/nvme/coco2017/train2017/')
conf.train_anno_path = Path(
    '/home/f/nvme/coco2017/annotations/instances_train2017.json')
conf.val_path = Path('/home/f/nvme/coco2017/val2017/')
conf.val_anno_path = Path(
    '/home/f/nvme/coco2017/annotations/instances_val2017.json')
conf.log_path = Path('/home/f/learning/yolo/log')
conf.model_path = Path('/home/f/learning/yolo/model')
conf.save_path = Path('/home/f/learning/yolo/save')
conf.ids_path = 'data/ids.npy'

conf.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

maps,correct_id_2_class = get_id_maps(conf)

loading annotations into memory...
Done (t=9.45s)
creating index...
index created!


In [4]:
conf.num_anchors = 3
conf.batch_size = 16
conf.input_size = 416
conf.scales = [32,16,8]

conf.running_norm = 0.
# conf.gdclip = 3000.
conf.num_workers = 8
conf.batch_size = 16
conf.gdclip = None
conf.coord_scale = 2.
conf.noobject_scale = 0.5
conf.object_scale = 5
conf.class_scale = 5.
conf.thresh = 0.5
conf.evaluate_iou_threshold = 0.5
conf.predict_confidence_threshold = 0.5

model = Yolo_model(conf)
model.to(conf.device)
conf.mean = model.res50_pyramid.model.mean
conf.std = model.res50_pyramid.model.std

conf.mse_loss = nn.MSELoss(size_average=False)
conf.bce_loss = nn.BCEWithLogitsLoss

conf.board_loss_every = 5
conf.evaluate_every = 5
conf.board_pred_image_every = 5
# conf.board_loss_every = len(train_loader) // 100
# conf.evaluate_every = len(train_loader) // 10
# conf.board_pred_image_every = len(train_loader) // 2
# conf.save_every = len(train_loader) // 2
# conf.board_grad_norm = len(train_loader) // 10

In [5]:
train_ds = Coco_dataset(conf,conf.train_path,conf.train_anno_path,maps)
val_ds = Coco_dataset(conf,conf.val_path,conf.val_anno_path,maps)
train_loader = DataLoader(train_ds,batch_size=conf.batch_size,shuffle=True,collate_fn=coco_collate_fn,pin_memory=True,num_workers=conf.num_workers)
val_loader = DataLoader(val_ds,batch_size=conf.batch_size,shuffle=False,collate_fn=coco_collate_fn,pin_memory=True,num_workers=conf.num_workers)

loading annotations into memory...
Done (t=9.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.30s)
creating index...
index created!


In [6]:
yolo = Yolo(conf,model,train_loader,val_loader,None,None)

In [7]:
imgs,bboxes_group,labels_group = next(iter(train_loader))

In [8]:
imgs = imgs.to(conf.device)
for i,label in enumerate(labels_group):
    labels_group[i] = label.to(conf.device)
for i,bboxes in enumerate(bboxes_group):
    bboxes_group[i] = bboxes.to(conf.device)

In [9]:
yolo.model.eval()

Yolo_model(
  (res50_pyramid): res50_pyramid(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(

In [10]:
o_ev = model(imgs)

> /root/Notebooks/YOLOv3/models/Yolo_model.py(117)forward()
-> xcyc = F.sigmoid(feat[:,:,:,:,:2])+ shift_xy.unsqueeze(0).unsqueeze(0)
(Pdb) l
112  	            if self.training:
113  	                loss_feats.append(feat)
114  	                feat = feat.detach().clone()
115  	
116  	            pdb.set_trace()
117  ->	            xcyc = F.sigmoid(feat[:,:,:,:,:2])+ shift_xy.unsqueeze(0).unsqueeze(0)
118  	            wh = torch.exp(feat[:,:,:,:,2:4]) * (anchors/scale).unsqueeze(1).unsqueeze(1).unsqueeze(0)
119  	            confidence = feat[:,:,:,:,4]
120  	            cls_pred = feat[:,:,:,:,5:]
121  	            if not self.training:
122  	                confi_group.append(confidence)
(Pdb) n
> /root/Notebooks/YOLOv3/models/Yolo_model.py(118)forward()
-> wh = torch.exp(feat[:,:,:,:,2:4]) * (anchors/scale).unsqueeze(1).unsqueeze(1).unsqueeze(0)
(Pdb) xcyc.shape
torch.Size([16, 3, 13, 13, 2])
(Pdb) xcyc[0,0,:,:,0]
tensor([[  0.5000,   1.5000,   2.5000,   3.5000,   4.5000,   5.500

(Pdb) l
113  	                loss_feats.append(feat)
114  	                feat = feat.detach().clone()
115  	
116  	            pdb.set_trace()
117  	            xcyc = F.sigmoid(feat[:,:,:,:,:2])+ shift_xy.unsqueeze(0).unsqueeze(0)
118  ->	            wh = torch.exp(feat[:,:,:,:,2:4]) * (anchors/scale).unsqueeze(1).unsqueeze(1).unsqueeze(0)
119  	            confidence = feat[:,:,:,:,4]
120  	            cls_pred = feat[:,:,:,:,5:]
121  	            if not self.training:
122  	                confi_group.append(confidence)
123  	                cls_pred_group.append(cls_pred)
(Pdb) F.sigmoid(feat[:,:,:,:,:2])[0,0,:,:,1]
tensor([[ 0.5000,  0.5000,  0.5000,  0.5000,  0.5000,  0.5000,  0.5000,
          0.5000,  0.5000,  0.5000,  0.5000,  0.5000,  0.5000],
        [ 0.5000,  0.5000,  0.5000,  0.5000,  0.5000,  0.5000,  0.5000,
          0.5000,  0.5000,  0.5000,  0.5000,  0.5000,  0.5000],
        [ 0.5000,  0.5000,  0.5000,  0.5000,  0.5000,  0.5000,  0.5000,
          0.5000,  0.5000

(Pdb) feat[:,:,:,:,:2][0,0,:,:,1]
tensor(1.00000e-05 *
       [[-0.9195, -0.6021, -0.8533, -0.9054, -0.9537, -0.8174, -0.3689,
         -0.2531, -0.1849, -0.0584, -0.2355, -0.3966, -0.1283],
        [-1.3600, -1.2226, -1.1653, -0.8471, -1.0331, -1.1814, -0.7535,
         -0.5488, -0.3409, -0.5629, -0.5891, -0.4702, -0.3012],
        [-1.0948, -1.4523, -1.6174, -1.3812, -1.1506, -0.7328, -0.8407,
         -0.7654, -0.5898, -0.3889, -0.0408, -0.3838, -0.2856],
        [-0.9271, -1.4217, -1.1408, -1.1645, -0.9548, -1.2033, -0.7622,
         -0.5933, -0.8372, -0.4706, -0.6657, -0.5846, -0.4454],
        [-0.6052, -0.9691, -0.7935, -1.1132, -0.9104, -0.6997, -0.5457,
         -1.0809, -1.5632, -1.2023, -1.1389, -0.5840, -0.7849],
        [-0.3584, -0.6629, -0.9299, -0.8078, -0.8143, -1.0041, -1.3943,
         -1.4081, -1.7457, -1.3066, -0.4770, -0.3480, -0.2411],
        [-0.4854, -0.6868, -1.3619, -1.1826, -0.9151, -1.5360, -1.7316,
         -2.6382, -2.4176, -1.8321, -1.4091, -0.3964, -0.

(Pdb) feat[:,:,:,:,2:4][0,1,:,:,0]
tensor(1.00000e-04 *
       [[ 0.3631,  0.8056,  0.7059,  0.3868,  0.4982,  0.7109,  0.2290,
         -0.0144,  0.0546,  0.0409,  0.0021, -0.0110, -0.0197],
        [ 0.5917,  1.0964,  0.6980,  0.6222,  0.3684,  0.6999,  0.3763,
          0.0873,  0.0910, -0.0065, -0.0264,  0.0176, -0.0168],
        [ 0.6433,  0.5717,  0.1073, -0.0171, -0.0161,  0.0350,  0.0860,
          0.1419, -0.0153, -0.0159,  0.0664,  0.0225, -0.0046],
        [ 0.5423,  0.9196,  0.2041, -0.0687, -0.0528, -0.0089,  0.1554,
          0.2629,  0.3071,  0.0695, -0.0271, -0.0334, -0.0151],
        [ 0.6608,  0.0878,  0.0497, -0.0291, -0.0142,  0.3013,  0.5302,
          0.5310, -0.0494, -0.0278, -0.0058, -0.0008, -0.0157],
        [ 0.0744, -0.0105,  0.3394, -0.0046, -0.0081,  0.3146,  0.7298,
         -0.0070, -0.0163, -0.0217,  0.4056, -0.0285, -0.0102],
        [-0.0113, -0.0134,  0.1536,  0.8586,  0.8114,  0.2502,  0.4347,
          0.6489,  1.1283,  0.4180, -0.0241,  0.3037,  0

BdbQuit: 

In [ ]:
o_ev.cls_pred_group[0].shape,o_ev.cls_pred_group[1].shape,o_ev.cls_pred_group[2].shape

In [ ]:
o_ev.confi_group[0].shape,o_ev.confi_group[1].shape,o_ev.confi_group[2].shape

In [ ]:
o_ev.pred_bboxes_group[0][0,1,:,:,1]

In [ ]:
o_ev.